LINKS:

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

---------

### Libraries

In [34]:
import os
import pandas as pd
import json

import openai
from openai import OpenAI
openai_api_key = os.environ.get("OPENAI_API_KEY")

### API OBJECT

In [35]:
client = OpenAI(api_key=openai_api_key)

---------

## Without function 
Example dummy function hard coded to return the same weather  
In production, this could be your backend API or an external API


In [49]:
messages = [{"role": "user", 
             "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]

In [37]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages
    # tools=tools,
    # tool_choice="auto"  # auto is default, but we'll be explicit
)
response.choices[0].message.content

'As of the latest update:\n\n- San Francisco, California: Cloudy with a high of 65°F and a low of 54°F. Chance of rain showers in the afternoon.\n- Tokyo, Japan: Mostly sunny with a high of 56°F and a low of 47°F. Light winds.\n- Paris, France: Cloudy with a high of 52°F and a low of 41°F. Light rain showers possible throughout the day.'

---------

## Function 

In [38]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

### Step 1: send the conversation and available functions to the model

In [39]:
tools = [{"type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {
                            "type": "string", 
                            "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

In [40]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
print(response.choices[0].message.content)

None


In [41]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jT1EgZ0kq8gE0nWvauGbWCZq', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nCZ2HPuP0t2RnMrfbdYhgpfQ', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_4Fr6WkvT1Oj7tALF3jvTrftd', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]))

In [42]:
response.choices[0].finish_reason

'tool_calls'

In [43]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_jT1EgZ0kq8gE0nWvauGbWCZq', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_nCZ2HPuP0t2RnMrfbdYhgpfQ', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_4Fr6WkvT1Oj7tALF3jvTrftd', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

In [44]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jT1EgZ0kq8gE0nWvauGbWCZq', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nCZ2HPuP0t2RnMrfbdYhgpfQ', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_4Fr6WkvT1Oj7tALF3jvTrftd', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')])

### Step 2: check if the model wanted to call a function

In [70]:
# Messages before
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
messages

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"}]

In [71]:
# Hacemos un append con la respuesta del LLM diciendo que tiene que utilizar herramientas
messages.append(response_message)  # extend conversation with assistant's reply
messages

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jT1EgZ0kq8gE0nWvauGbWCZq', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_nCZ2HPuP0t2RnMrfbdYhgpfQ', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_4Fr6WkvT1Oj7tALF3jvTrftd', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')])]

In [72]:
# Step 3: call the function
# Note: the JSON response may not always be valid; be sure to handle errors
available_functions = {
    "get_current_weather": get_current_weather,
}  # only one function in this example, but you can have multiple


---------

---------

# FUNCTION 2

In [97]:
student_description = "sunny savita is a student of computer science at IIT delhi. He is an indian and has a 8.5 GPA. Sunny is known for his programming skills and is an active member of the college's AI Club. He hopes to pursue a career in artificial intelligence after graduating."

In [98]:
# A simple prompt to extract information from "student_description" in a JSON format.
prompt = f'''
Please extract the following information from the given text and return it as a JSON object:

name
college
grades
club

This is the body of text to extract the information from:
{student_description}
'''

In [99]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": prompt
    }
      ]
)

In [100]:
response.choices[0].message.content

'{\n  "name": "sunny savita",\n  "college": "IIT delhi",\n  "grades": 8.5,\n  "club": "AI Club"\n}'

In [101]:
json.loads(response.choices[0].message.content)

{'name': 'sunny savita',
 'college': 'IIT delhi',
 'grades': 8.5,
 'club': 'AI Club'}

### Now lets use a function

In [102]:
student_custom_function = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'college': {
                    'type': 'string',
                    'description': 'The college name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'CGPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'college club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [106]:
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt }],
    functions=student_custom_function
)

In [104]:
response2.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{}', name='extract_student_info'), tool_calls=None)

In [105]:
response2.choices[0].message.function_call

FunctionCall(arguments='{}', name='extract_student_info')

In [107]:
json.loads(response2.choices[0].message.function_call.arguments)

{}

In [108]:
student_description_two="krish naik is a student of computer science at IIT Mumbai. He is an indian and has a 9.5 GPA. krish is known for his programming skills and is an active member of the college's data science Club. He hopes to pursue a career in artificial intelligence after graduating."

In [109]:
student_description_three="sudhanshu kumar is a student of computer science at IIT bengalore. He is an indian and has a 9.2 GPA. krish is known for his programming skills and is an active member of the college's MLops Club. He hopes to pursue a career in artificial intelligence after graduating."

In [111]:
import json
student_info = [student_description, student_description_two,student_description_three]
for student in student_info:
    response =  client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': student}],
        functions = student_custom_function,
        function_call = 'auto'
    )

    response = json.loads(response.choices[0].message.function_call.arguments)
    print(response)#import csv

{}
{}
{'name': 'sudhanshu kumar', 'college': 'IIT bangalore', 'grades': 92, 'club': 'MLops Club'}


In [114]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": "When's the next flight from delhi to mumbai?"
    }
      ]
)
response.choices[0].message.content

'I am unable to provide real-time flight information. It is recommended to check with airlines or travel websites for the most up-to-date flight schedules.'

In [115]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DEL",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. MUM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

In [116]:
user_prompt = "When's the next flight from new delhi to mumbai?"

In [118]:
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": user_prompt
    }
      ],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
    
)
response2.choices[0].message.content

In [120]:
response.choices[0].message

ChatCompletionMessage(content='I am unable to provide real-time flight information. It is recommended to check with airlines or travel websites for the most up-to-date flight schedules.', role='assistant', function_call=None, tool_calls=None)